### Welcome to GS Python Editor
* Code in the cell below is generated and updated by GS. Please do not modify them unless you changed your function name or signature.
    * During execution, GS calls \_\_gsWrapper\_\_, which in turn calls your function. If you change your function names or singature, please make sure the calling code in \_\_gsWrapper\_\_ is correct.
* Gid of this function: DACCFABAD78F4936AA0B527FEDC0432B

In [1]:
from lib.gftTools import gftIO

def __gsWrapper__(context,target_mode,position_limit,risk_model,asset_return,asset_weight,target_risk,target_return,target_date,asset_constraint,group_constraint,exposure_constraint,lambda_risk,beta_tranaction,alpha_return):
    return CVXOptimizerBnd(context,target_mode,position_limit,risk_model,asset_return,asset_weight,target_risk,target_return,target_date,asset_constraint,group_constraint,exposure_constraint,lambda_risk,beta_tranaction,alpha_return)

### Save and Publish
 * Click the "save" button (at the left end of the toolbar) will save the notebook; the python file with same filename in the home directory will be updated too.
    * Code in all code cells not containing string "%debug" or "debug\_\_gsWrapper\_\_" will be saved to the python file.
    * Please leave the auto-generated comments in python file unchanged. They are used to recreate notebook from python file.
* Click the "publish" button (at the right end of the toolbar) to publish the saved python file to GS.
    * If the notebook is not saved, click publish will save it and update the python file, then publish.

### Start Coding Here
* If you are debugging you code, please find the entry function for debug at the bottom of the cells.
    * Click menu item "Cell / Run All" to call out ipdb and step into debug entry function.
* All cells below the debug entry function cell will not be saved to python file.

In [4]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import re

from cvxopt import matrix, solvers, spmatrix, sparse
from cvxopt.blas import dot

from lib.gftTools import gsConst
# import U_PNL_FITNESS as fitness

solvers.options['show_progress'] = False




def logrels(rets):
    """Log of return relatives, ln(1+r), for a given DataFrame rets."""
    return np.log(rets + 1)


def get_ret_range(rets, df_asset_bound):
    ''' Calculate theoretical minimum and maximum theoretical returns.

    Parameters
    ----------
    rets: dataframe

    df_asset_bound : dataframe-like
        Input lower and upper boundary dataframe for each asset.

    Returns
    -------
    (f_min, f_max): tuple
    '''
    from copy import deepcopy
    f_min = 0
    f_max = 0

    rets = deepcopy(rets)

#    na_expected = np.average(rets, axis=0)
    na_expected = logrels(rets).mean().values

    na_signs = np.sign(na_expected)
    indices = np.where(na_signs == 0)
    na_signs[indices] = 1
    na_signs = np.ones(len(na_signs))

    rets = na_signs*rets
    na_expected = na_signs*na_expected

    na_sort_ind = na_expected.argsort()

    # First add the lower bounds on portfolio participation
    for i, fRet in enumerate(na_expected):
        f_min = f_min + fRet*df_asset_bound.lower[i]
        f_max = f_max + fRet*df_asset_bound.lower[i]


    # Now calculate minimum returns
    # allocate the max possible in worst performing equities
    # Subtract min since we have already counted it
    na_upper_add = df_asset_bound.upper - df_asset_bound.lower
    f_total_weight = np.sum(df_asset_bound.lower)

    for i, ls_ind in enumerate(na_sort_ind):
        f_ret_add = na_upper_add[ls_ind] * na_expected[ls_ind]
        f_total_weight = f_total_weight + na_upper_add[ls_ind]
        f_min = f_min + f_ret_add
        # Check if this additional percent puts us over the limit
        if f_total_weight > 1.0:
            f_min = f_min - na_expected[ls_ind] * (f_total_weight - 1.0)
            break
    else:
        raise ValueError("sum of total asset maximum weight is less than 1 ")
    # Repeat for max, just reverse the sort, i.e. high to low
    na_upper_add = df_asset_bound.upper - df_asset_bound.lower
    f_total_weight = np.sum(df_asset_bound.lower)
    if f_total_weight > 1:
        raise ValueError("sum of total asset minimum weight is bigger than 1 ")
    for i, ls_ind in enumerate(na_sort_ind[::-1]):
        f_ret_add = na_upper_add[ls_ind] * na_expected[ls_ind]
        f_total_weight = f_total_weight + na_upper_add[ls_ind]
        f_max = f_max + f_ret_add

        # Check if this additional percent puts us over the limit
        if f_total_weight > 1.0:
            f_max = f_max - na_expected[ls_ind] * (f_total_weight - 1.0)
            break

    return (f_min, f_max)


def check_boundary_constraint(df_asset_bound, df_group_bound,
                              df_exposure_bound, df_exposure):
    ''' check input boundary limit.

    Parameters
    ----------
    df_asset_bound : dataframe-like
        Input lower and upper boundary dataframe for each asset.

    df_group_bound : dataframe-like
        Input lower and upper boundary dataframe for each group.

    df_exposure_bound : dataframe-like
        Input lower and upper boundary dataframe for each factor.

    df_exposure : dataframe
        Big X.

    Returns
    -------
    True: all boundaries in condition.
    False: any boundaries out of condition.
    '''
    if ((df_asset_bound.lower) < 0).any():
        raise ValueError('short is not supported.')
    if ((df_asset_bound.upper) > 1).any():
        raise ValueError('asset upper boundary is bigger than 1.')
    if (np.sum(df_asset_bound.lower) > 1):
        raise ValueError('asset lower boundary sum is bigger than 1.')
    if (np.sum(df_asset_bound.upper) < 1):
        raise ValueError('asset upper boundary sum is smaller than 1.')
    if ((df_asset_bound.lower > df_asset_bound.upper).any()):
        raise ValueError('asset lower boundary is bigger than upper boundary')

    if ((df_group_bound.lower) < 0).any():
        raise ValueError('short is not supported.')
    if ((df_group_bound.upper) > 1).any():
        raise ValueError('group upper boundary is bigger than 1.')
    if (np.sum(df_group_bound.lower) > 1):
        raise ValueError('group lower boundary sum is bigger than 1.')
    if (np.sum(df_group_bound.upper) < 1):
        raise ValueError('group upper boundary sum is smaller than 1.')
    if ((df_group_bound.lower > df_group_bound.upper).any()):
        raise ValueError('group lower boundary is bigger than upper boundary')

    df_factor_exposure_bound_check = pd.DataFrame(index=df_exposure.T.index,
                                                  columns=[['lower', 'upper']])
    df_factor_exposure_bound_check.lower = df_exposure.T.min(axis=1)
    df_factor_exposure_bound_check.upper = df_exposure.T.max(axis=1)

    if (df_factor_exposure_bound_check.upper < df_exposure_bound.upper).any():
        raise ValueError('factor exposure upper setting error')

    if (df_factor_exposure_bound_check.lower > df_exposure_bound.lower).any():
        raise ValueError('factor exposure lower setting error')

    return True


class ConstraintError(Exception):
    pass


def check_constraint_issue(mx_P, mx_q, mx_G, mx_h, mx_A, mx_b,
                           mx_asset_sub, mx_group_sub, mx_exp_sub,
                           mx_asset_bnd, mx_group_bnd, mx_exp_bnd):
    ''' check which constraint fails.

    Parameters
    ----------
    mx_G: 1xn matrix
        matrix only includes returns.
    mx_h: 1x1 matrix
        target return matrix.
    mx_asset_sub: 2nxn subsets matrix, n=asset number
        2 nx1 identity matrices.
    mx_group_sub: 2axn subsets matrix, a=group number
        2 axn spmatrices.
    mx_exp_sub: 2bxn subsets matrix, b=exposure factors number
        2bxn identity matrices.
    mx_asset_bnd: 2nx1 matrix, n=asset number
        asset weight constraint matrix
    mx_group_bnd: 2ax1 matrix, a=group number
        group weight constraint matrix
    mx_exp_bnd: 2bx1 matrix, b=exposure factors number
        factor exposure constraint matrix

    Returns
    -------
    None
    '''
    import itertools
    G = matrix(sparse([mx_G, mx_asset_sub]))
    h = matrix(sparse([mx_h, mx_asset_bnd]))

    boundary_sub = [mx_group_sub, mx_exp_sub]
    limit = [mx_group_bnd, mx_exp_bnd]
    error = ('group ', 'exposure ')
    stuff = [1, 2]
    for L in range(0, len(stuff)+1):
        for subset in itertools.combinations(stuff, L):
            if len(subset) == 0:
                try:
                    # G_pos = matrix(sparse([G, matrix(-np.eye(n), tc='d')]))
                    # h_pos = matrix(sparse([h, matrix(np.zeros((n, 1)))]))
                    sol = solvers.qp(mx_P, mx_q, G, h, mx_A, mx_b)
                    if sol['x'] == 'unknown':
                        raise('failed to get optimal value on\
                        position limit constraint')
                except ValueError as e:
                    raise ConstraintError('ERROR on solving position limit\
                    constraint only')

            if len(subset) > 0:
                ls = [x-1 for x in list(subset)]
                g_matrix = []
                h_matrix = []
                g_matrix.append(G)
                h_matrix.append(h)
                for i in ls:
                    g_matrix.append(boundary_sub[i])
                    h_matrix.append(limit[i])

                G_val = matrix(sparse(g_matrix))
                h_val = matrix(sparse(h_matrix))

                try:
                    sol = solvers.qp(mx_P, mx_q, G_val, h_val, mx_A, mx_b)
                    if sol['x'] == 'unknown':
                        print('failed to get optimal value on %s', [error[i] for i in ls])
                except ValueError as e:
                    raise ConstraintError('ERROR on solving combination %s, %s' % ([error[i] for i in ls], e))

    return True

def statistics(weights, rets, covariance):
    """Compute expected portfolio statistics from individual asset returns.

    Parameters
    ----------
    rets : DataFrame
        Individual asset returns.  Use numeral rather than decimal form
    weights : array-like
        Individual asset weights, nx1 vector.

    Returns
    -------
    list of (pret, pvol, pstd); these are *per-period* figures (not annualized)
        pret : expected portfolio return
        pvol : expected portfolio variance
        psr  : sharpe ratio

    """

    if isinstance(weights, (tuple, list)):
        weights = np.array(weights)

    if isinstance(weights, matrix):
        pret = np.sum(logrels(rets).mean().values * weights.T)
        pvol = np.dot(weights.T, np.dot(covariance, weights))
    elif isinstance(weights, pd.DataFrame):
        pret = np.dot(weights.values, logrels(rets).mean().T)
        pvol = np.dot(weights, np.dot(covariance, weights.T))
    pstd = np.sqrt(pvol)

    return [pret, pvol, pret/pstd]


def get_factor_exposure(risk_model, factor_list, date, symbols):
    ''' Return factor exposure matrix(big X).

    Parameters
    ----------
    risk_model: dictionary
        Including specific risk, different factor exposure dataframe for all
        symbols.

    factor_list: list
        Factor exposure list.

    Returns
    -------
    factor_exposure: DataFrame
        Big X on target date for input symbols.
    '''
    factor_exposure = pd.DataFrame(index=symbols)
    for factor in factor_list:
        try:
            factor_exposure[factor] = risk_model[factor].asMatrix().\
                                      loc[date, symbols]
        except KeyError:
            raise KeyError('invalid input date: %s' % date)
    factor_exposure.columns = gftIO.strSet2Np(factor_exposure.columns.values)
    factor_exposure = factor_exposure.fillna(0)

    return factor_exposure


def find_nearest(array, value):
    """ Find the nearest value index from an array"""
    if isinstance(array, list):
        array = np.array(array)
    idx = (np.abs(array-value)).argmin()
    return idx


def CVXOptimizerBnd(context, target_mode, position_limit, risk_model,
                    asset_return, asset_weight, target_risk,
                    target_return, target_date, asset_constraint,
                    group_constraint, exposure_constraint,
                    lambda_risk,beta_tranaction,alpha_return):
    """
    optimize fund weight target on different constraints, objective, based on
    target type and mode, fund return target, fund weight, group weight， etc.

    Parameters
    ----------
    target_date: Timestamp
        Specific date.

    target_mode: dictionary
        target optimization type({type: mode})
        0: minimum risk.
        1: minimum risk subject to target return.
        2: maximum return subject to target risk.

    asset_return: Dataframe, OTV,
        asset return for all symbols.
        index=date, O: asset names, V: asset return.

    risk model: dictionary
        Risk factor exposure: DataFrame
            所有股票在因子上暴露的值，p.s. 如有8个因子，就有8个DataFrame,
            得把所有8个因子某一天所有值先取出来得到一个n*k的矩阵.n为股票，k为因子
        Specific Risk: DataFrame
            用来组成对角矩阵Delta.

    asset_weight: Dataframe, OOTV
        T=date, O: asset names, O: group names, V: asset weight.
        weight bound of each asset. Default is equal weight.

    target_return: double
        Target return for portfolio respected to benchmark.

    target_risk: double
        Portfolio risk tolerance whose objective is maximum return.

    Returns:
    ----------
    df_result: DataFrame
        Optimized value of weight.
        Index: target date.
        Columns: assets names.

    """
    import logging
    logger = logging.getLogger()
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.DEBUG)

    asset_return = asset_return.asMatrix()
    asset_weights = asset_weight.asColumnTab()
    target_date = pd.to_datetime(target_date)
    target_return = target_return * alpha_return
    target_risk = target_risk * lambda_risk
    if asset_constraint is not None:
        asset_constraint = asset_constraint.asMatrix()
    if group_constraint is not None:
        group_constraint = group_constraint.asMatrix()
    if exposure_constraint is not None:
        exposure_constaint = exposure_constraint.asMatrix()

    logger.debug('parse data finished!')
    logger.debug('asset return number: %s', asset_return.shape[1])
    logger.debug('asset weight number: %s', asset_weights.shape[0])
    logger.debug('parse data finished.')
    # regex to search all the factors
    ls_factor = [x[:-4] for x in risk_model.keys() if re.search(".ret$", x)]
    # ls_factor = [x.split('.')[0] for x in ls_factor]

    specific_risk = risk_model['specificRisk'].pivot(
        index='date', columns='symbol', values='specificrisk')
    # target_date = pd.datetime(year=2016, month=10, day=31)
    # target_return = -0.00096377
    # target_risk = 3.16026352e-06
    #target_mode = 1
    #position_limit = 500

    # find the nearest date next to target date from specific risk
    dt_next_to_target = specific_risk.index.searchsorted(target_date)
    dt_next_to_target = specific_risk.index[dt_next_to_target]
    target_specific_risk = specific_risk.loc[dt_next_to_target, :]
    logger.debug('target date: %s', target_date)
    logger.debug('next to target date: %s', dt_next_to_target)
    # drop duplicated rows at date
    df_industries_asset_weight = asset_weights.drop_duplicates(
        subset=['date', 'symbol'])
    try:
        df_industries_asset_init_weight = df_industries_asset_weight[
            df_industries_asset_weight['date'] == target_date].dropna()
    except KeyError:
        raise KeyError('invalid input date: %s' % target_date)

    # drop incomplete rows
    df_industries_asset_init_weight = df_industries_asset_init_weight.dropna(
        axis=0, subset=['industry', 'symbol'], how='any')

    unique_symbol = df_industries_asset_init_weight['symbol'].unique()
    target_symbols = target_specific_risk.index.intersection(unique_symbol)
    if position_limit > len(target_symbols):
        logger.debug("position limit is bigger than total symbols.")
        position_limit = len(target_symbols)

    # get random symbols at the target position limit
    arr = list(range(len(target_symbols)))
    np.random.shuffle(arr)
    target_symbols = target_symbols[arr[:position_limit]]

    df_industries_asset_target_init_weight = df_industries_asset_init_weight.\
                                             loc[df_industries_asset_init_weight['symbol'].isin(target_symbols)]
    df_pivot_industries_asset_weights = pd.pivot_table(
        df_industries_asset_target_init_weight, values='value', index=['date'],
        columns=['industry', 'symbol'])
    df_pivot_industries_asset_weights = df_pivot_industries_asset_weights.fillna(0)
    logger.debug("set OOTV to hierachical index dataframe.")
    noa = len(target_symbols)
    if noa < 1:
        raise ValueError("no intersected symbols from specific risk and initial holding.")

    # get the ordered column list
    idx_level_0_value = df_pivot_industries_asset_weights.columns.get_level_values(0)
    idx_level_0_value = idx_level_0_value.drop_duplicates()
    idx_level_1_value = df_pivot_industries_asset_weights.columns.get_level_values(1)
    asset_return = asset_return.loc[:target_date, idx_level_1_value].fillna(0)

    diag = specific_risk.loc[dt_next_to_target, idx_level_1_value]
    delta = pd.DataFrame(np.diag(diag), index=diag.index,
                         columns=diag.index).fillna(0)

    big_X = get_factor_exposure(risk_model, ls_factor, target_date,
                                idx_level_1_value)
    big_X = big_X.fillna(0)
    all_factors = big_X.columns

    covariance_matrix = risk_model['ret_cov'].set_index('date')

    cov_matrix = covariance_matrix.loc[dt_next_to_target]
    cov_matrix = cov_matrix.pivot(index='factorid1', columns='factorid2',
                                  values='value')
    cov_matrix = cov_matrix.reindex(all_factors, all_factors, fill_value=np.nan)

    cov_matrix_V = big_X.dot(cov_matrix).dot(big_X.T) + delta

    P = matrix(cov_matrix_V.values)
    q = matrix(np.zeros((noa, 1)), tc='d')

    A = matrix(1.0, (1, noa))
    b = matrix(1.0)

    # for group weight constraint
    groups = df_pivot_industries_asset_weights.groupby(
        axis=1, level=0, sort=False, group_keys=False).count().\
        iloc[-1, :].values
    num_group = len(groups)
    num_asset = np.sum(groups)

    logger.debug('number of assets in groups: %s', num_asset)
    logger.debug('number of groups: %s', num_group)


    # set boundary vector for h
    df_asset_weight = pd.DataFrame({'lower': [0.0], 'upper': [1.0]},
                                   index=idx_level_1_value)
    df_group_weight = pd.DataFrame({'lower': [0.0], 'upper': [1.0]},
                                   index=idx_level_0_value)
    df_factor_exposure_bound = pd.DataFrame(index=big_X.T.index, columns=[['lower', 'upper']])
    df_factor_exposure_bound.lower = (1.0/noa)*big_X.sum()*(0.999991)
    df_factor_exposure_bound.upper = (1.0/noa)*big_X.sum()*(1.000009)

    if asset_constraint is not None:
        try:
            df_asset_weight.lower.ix[asset_constraint.lower.index] = asset_constraint.lower
            df_asset_weight.upper.ix[asset_constraint.upper.index] = asset_constraint.upper
        except KeyError:
            raise('input target asset is not in the initial asset.')
    if group_constraint is not None:
        try:
            df_group_weight.lower.ix[group_constraint.lower.index] = group_constraint.lower
            df_group_weight.upper.ix[group_constraint.upper.index] = group_constraint.upper
        except KeyError:
            raise('input target group is not in the initial group.')

    if exposure_constraint is not None:
        try:
            df_factor_exposure_bound.lower.ix[exposure_constraint.lower.index] = exposure_constraint.lower
            df_factor_exposure_bound.upper.ix[exposure_constraint.upper.index] = exposure_constraint.upper
        except KeyError:
            raise('input target factor is not possible.')

    if check_boundary_constraint(df_asset_weight, df_group_weight,
                                 df_factor_exposure_bound, big_X):
        logger.debug("boundary setting is fine")

    df_asset_bnd_matrix = matrix(np.concatenate(((df_asset_weight.upper,
                                                  df_asset_weight.lower)), 0))
    df_group_bnd_matrix = matrix(np.concatenate(((df_group_weight.upper,
                                                  df_group_weight.lower)), 0))
    df_factor_exposure_bnd_matrix = matrix(np.concatenate(((df_factor_exposure_bound.upper,
                                                            df_factor_exposure_bound.lower)), 0))

    # Assuming AvgReturns as the expected returns if parameter is not specified
    rets_mean = logrels(asset_return).mean()
    avg_ret = matrix(rets_mean.values)
    G = matrix(-np.transpose(np.array(avg_ret)))
    h = matrix(-np.ones((1, 1))*target_return)
    G_sparse_list = []
    for i in range(num_group):
        for j in range(groups[i]):
            G_sparse_list.append(i)
    Group_sub = spmatrix(1.0, G_sparse_list, range(num_asset))

    Group_sub = matrix(sparse([Group_sub, -Group_sub]))

    asset_sub = matrix(np.eye(noa))
    asset_sub = matrix(sparse([asset_sub, -asset_sub]))
    exp_sub = matrix(np.array(big_X.T))
    exp_sub = matrix(sparse([exp_sub, - exp_sub]))

    # minimum risk
    if target_mode == 0:
        # G = matrix(-np.eye(noa), tc='d')
        # h = matrix(-np.zeros((noa, 1)), tc='d')
        if exposure_constraint is not None:
            G0 = matrix(sparse([asset_sub, Group_sub, exp_sub]))
            h0 = matrix(sparse([df_asset_bnd_matrix, df_group_bnd_matrix,
                               df_factor_exposure_bnd_matrix]))
        else:
            G0 = matrix(sparse([asset_sub, Group_sub]))
            h0 = matrix(sparse([df_asset_bnd_matrix, df_group_bnd_matrix]))

        try:
            sol = solvers.qp(P, q, G0, h0, A, b)
        except ValueError:
            h = matrix(-np.ones((1, 1))*100.0)
            check_constraint_issue(P, q, G, h, A, b, asset_sub, Group_sub,
                                   exp_sub, df_asset_bnd_matrix,
                                   df_group_bnd_matrix,
                                   df_factor_exposure_bnd_matrix)
        df_opts_weight = pd.DataFrame(np.array(sol['x']).T,
                                      columns=target_symbols,
                                      index=[target_date])
    # minimum risk subject to target return, Markowitz Mean_Variance Portfolio
    elif target_mode == 1:
        (f_min, f_max) = get_ret_range(asset_return, df_asset_weight)

        if target_return < f_min or target_return > f_max:
            raise ValueError("target return not possible")
        if exposure_constraint is not None:
            G1 = matrix(sparse([G, asset_sub, Group_sub, exp_sub]))
            h1 = matrix(sparse([h, df_asset_bnd_matrix, df_group_bnd_matrix,
                               df_factor_exposure_bnd_matrix]))
        else:
            G1 = matrix(sparse([G, asset_sub, Group_sub]))
            h1 = matrix(sparse([h, df_asset_bnd_matrix, df_group_bnd_matrix]))
        try:
            sol = solvers.qp(P, q, G1, h1, A, b)
        except ValueError:
            check_constraint_issue(P, q, G, h, A, b, asset_sub, Group_sub,
                                   exp_sub, df_asset_bnd_matrix,
                                   df_group_bnd_matrix,
                                   df_factor_exposure_bnd_matrix)

        df_opts_weight = pd.DataFrame(np.array(sol['x']).T,
                                      columns=target_symbols,
                                      index=[target_date])
    # Computes a tangency portfolio, i.e. a maximum Sharpe ratio portfolio
    elif target_mode == 2:
        (f_min, f_max) = get_ret_range(asset_return, df_asset_weight)
        N = 100
        f_step = (f_max - f_min) / N
        ls_f_return = [f_min + x * f_step for x in range(N + 1)]
        ls_f_risk = []
        ls_portfolio = []
        ls_f_result = []
        for i, f_target_return in enumerate(ls_f_return):
            logger.debug('target return: %s %s', i, f_target_return)
            h = matrix(-np.ones((1, 1))*f_target_return)
            if exposure_constraint is not None:
                G_sr = matrix(sparse([G, asset_sub, Group_sub, exp_sub]))
                h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                                      df_group_bnd_matrix,
                                      df_factor_exposure_bnd_matrix]))
            else:
                G_sr = matrix(sparse([G, asset_sub, Group_sub]))
                h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                                      df_group_bnd_matrix]))
            try:
                sol = solvers.qp(P, q, G_sr, h_sr, A, b)
                logger.debug("solution is: %s", sol['status'])
            except ValueError:
                ls_f_risk.append(np.nan)
                ls_portfolio.append(None)
                continue
            f_result = statistics(sol['x'], asset_return, cov_matrix_V)
            ls_f_result.append(f_result)
            logger.debug('target result calculated from weight: %s', f_result)

            ls_f_risk.append(statistics(sol['x'], asset_return, cov_matrix_V)[1])
            df_opts_weight = pd.DataFrame(np.array(sol['x']).T,
                                          columns=target_symbols,
                                          index=[target_date])
            ls_portfolio.append(df_opts_weight)

        ls_f_return_new = np.array(ls_f_return)
        ls_f_risk_new = np.array(ls_f_risk)
        ls_f_risk_new = ls_f_risk_new[ls_f_risk_new <= target_risk]
        if len(ls_f_risk_new) == 0:
            raise ValueError("target risk is not possible")
        ls_f_return_new = ls_f_return_new[ls_f_risk_new <= target_risk]
        na_sharpe_ratio = np.array(ls_f_result)[:, 2]
        logger.debug("maximum sharpe ratio is: %s", max(na_sharpe_ratio))
        i_index_max_sharpe = np.where(na_sharpe_ratio == max(na_sharpe_ratio))
        i_index_max_sharpe = i_index_max_sharpe[0]
        f_target = ls_f_return_new[i_index_max_sharpe]
        h = matrix(-np.ones((1, 1))*f_target)
        logger.debug("maximum sharpe ratio index is: %s", i_index_max_sharpe)
        logger.debug("maximum sharpe ratio return is: %s", f_target)
        logger.debug("maximum sharpe ratio risk is: %s", ls_f_risk_new[i_index_max_sharpe])

        if exposure_constraint is not None:
            G_sr = matrix(sparse([G, asset_sub, Group_sub, exp_sub]))
            h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                                  df_group_bnd_matrix,
                                  df_factor_exposure_bnd_matrix]))
        else:
            G_sr = matrix(sparse([G, asset_sub, Group_sub]))
            h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                                  df_group_bnd_matrix]))
        sol = solvers.qp(P, q, G_sr, h_sr, A, b)
        logger.debug("maximum sharpe ratio solution is %s", sol['status'])
        df_opts_weight = pd.DataFrame(np.array(sol['x']).T,
                                      columns=target_symbols,
                                      index=[target_date])

    # if sol['status'] == 'optimal':
    #     logger.debug('result is optimal')
    # elif sol['status'] == 'unknown':
    #     logger.warn('Convergence problem, the algorithm failed to find a solution that satisfies the specified tolerances')

    return df_opts_weight


In [4]:
from lib.gftTools import gftIO
def debug__gsWrapper__():
    context = gftIO.zload("/home/gft/data/context.pkl")
    x0 = gftIO.zload("/home/gft/data/x0.pkl")
    x1 = gftIO.zload("/home/gft/data/x1.pkl")
    x2 = gftIO.zload("/home/gft/data/x2.pkl")
    x3 = gftIO.zload("/home/gft/data/x3.pkl")
    x4 = gftIO.zload("/home/gft/data/x4.pkl")
    x5 = gftIO.zload("/home/gft/data/x5.pkl")
    x6 = gftIO.zload("/home/gft/data/x6.pkl")
    x7 = gftIO.zload("/home/gft/data/x7.pkl")
    x8 = gftIO.zload("/home/gft/data/x8.pkl")
    x9 = gftIO.zload("/home/gft/data/x9.pkl")
    x10 = gftIO.zload("/home/gft/data/x10.pkl")
    x11 = gftIO.zload("/home/gft/data/x11.pkl")
    x12 = gftIO.zload("/home/gft/data/x12.pkl")
    x13 = gftIO.zload("/home/gft/data/x13.pkl")
    __gsWrapper__(context, x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13)

In [ ]:
%debug debug__gsWrapper__()

# debug whole

In [5]:
context = gftIO.zload("/home/gft/data/context.pkl")
x0 = gftIO.zload("/home/gft/data/x0.pkl")
x1 = gftIO.zload("/home/gft/data/x1.pkl")
x2 = gftIO.zload("/home/gft/data/x2.pkl")
x3 = gftIO.zload("/home/gft/data/x3.pkl")
x4 = gftIO.zload("/home/gft/data/x4.pkl")
x5 = gftIO.zload("/home/gft/data/x5.pkl")
x6 = gftIO.zload("/home/gft/data/x6.pkl")
x7 = gftIO.zload("/home/gft/data/x7.pkl")
x8 = gftIO.zload("/home/gft/data/x8.pkl")
x9 = gftIO.zload("/home/gft/data/x9.pkl")
x10 = gftIO.zload("/home/gft/data/x10.pkl")
x11 = gftIO.zload("/home/gft/data/x11.pkl")
x12 = gftIO.zload("/home/gft/data/x12.pkl")
x13 = gftIO.zload("/home/gft/data/x13.pkl")
result = __gsWrapper__(context, 0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13)
result

2017-07-03 05:55:03,933 root         DEBUG    parse data finished!
2017-07-03 05:55:03,972 root         DEBUG    asset return number: 3363
2017-07-03 05:55:03,977 root         DEBUG    asset weight number: 10540
2017-07-03 05:55:03,979 root         DEBUG    parse data finished.
2017-07-03 05:55:06,609 root         DEBUG    target date: 2016-10-31 00:00:00
2017-07-03 05:55:06,612 root         DEBUG    next to target date: 2016-10-31 00:00:00
2017-07-03 05:55:06,660 root         DEBUG    set OOTV to hierachical index dataframe.
2017-07-03 05:55:07,039 root         DEBUG    number of assets in groups: 500
2017-07-03 05:55:07,043 root         DEBUG    number of groups: 28
2017-07-03 05:55:07,069 root         DEBUG    boundary setting is fine


,"b'\xe2\x11\xeaH\x11 \x01\xc1""\'i\xcb+xu\x90'","b'\xe2\x11\xeaH\x7f\xfd#\xc1""\'i\xcb+xu\x90'","b'\xe2\x11\xeaHN\xcf\x05\xc1""\'i\xcb+xu\x90'","b'\xe2\x11\xeaH \xf1\x1b\xc1""\'i\xcb+xu\x90'","b'\xe2\x11\xeaH\x9c&?\xc1""\'i\xcb+xu\x90'","b'\xe2\x11\xeaH\x00\xd3=\xc1""\'i\xcb+xu\x90'",b'WF\t`l\xf9\xe2\xf0\x8d\x8ba\x15\x1d\x8cV\xa6',"b'\xe2\x11\xeaH3\x9f$\xc1""\'i\xcb+xu\x90'","b'\xe2\x11\xeaH[X!\xc1""\'i\xcb+xu\x90'","b'\xe2\x11\xeaH*A*\xc1""\'i\xcb+xu\x90'",...,"b'\xe2\x11\xeaH\xbbC@\xc1""\'i\xcb+xu\x90'","b'\xe2\x11\xeaH\xde\xe2<\xc1""\'i\xcb+xu\x90'","b'\xe2\x11\xeaH\xbbqA\xc1""\'i\xcb+xu\x90'","b'\xe2\x11\xeaHe\xf6;\xc1""\'i\xcb+xu\x90'","b'\xe2\x11\xeaH\x16\xc3:\xc1""\'i\xcb+xu\x90'","b'\xe2\x11\xeaH\xc7\xd1B\xc1""\'i\xcb+xu\x90'","b'\xe2\x11\xeaH\xd5h\x1b\xc1""\'i\xcb+xu\x90'","b'\xe2\x11\xeaHK\x0b\x1c\xc1""\'i\xcb+xu\x90'","b'\xe2\x11\xeaH\xc3\xfb=\xc1""\'i\xcb+xu\x90'","b'\xe2\x11\xeaH\x85\x88.\xc1""\'i\xcb+xu\x90'"
2016-10-31,0.000402,0.000786,0.000838,0.000525,0.000374,0.000615,0.00047,0.000526,0.001319,0.000233,...,0.00029,0.000294,0.022149,0.000337,0.001038,0.000784,0.000707,0.001834,0.001143,0.01574


In [ ]:
result[result<0].sum(1)

In [ ]:
(result > 0).all().all()

In [ ]:
(result <= 1).all().all()

In [ ]:
(result[result>0]).sum(1)

In [ ]:
import matplotlib.pyplot as plt
plt.show()

# debug part by part

In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import re
import os

from cvxopt import matrix, solvers, spmatrix, sparse
from cvxopt.blas import dot

from lib.gftTools import gsConst, gftIO
# import U_PNL_FITNESS as fitness

solvers.options['show_progress'] = False

import logging
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

path = '/home/weiwu/share/optimize/'

alpha_return = gftIO.zload(os.path.join(path, 'alpha_return.pkl'))
asset_constraint = gftIO.zload(os.path.join(path, 'asset_constraint.pkl'))
asset_return = gftIO.zload(os.path.join(path, 'asset_return.pkl'))
asset_weight = gftIO.zload(os.path.join(path, 'asset_weight.pkl'))
beta_transaction = gftIO.zload(os.path.join(path, 'beta_transaction.pkl'))
exposure_constraint = gftIO.zload(os.path.join(path, 'exposure_constraint.pkl'))
factor_exposure = gftIO.zload(os.path.join(path, 'factor_exposure.pkl'))
group_constraint = gftIO.zload(os.path.join(path, 'group_constraint.pkl'))
lambda_risk = gftIO.zload(os.path.join(path, 'lambda_risk.pkl'))
position_limit = gftIO.zload(os.path.join(path, 'position_limit.pkl'))
risk_model = gftIO.zload(os.path.join(path, 'risk_model.pkl'))
target_date = gftIO.zload(os.path.join(path, 'target_date.pkl'))
target_mode = gftIO.zload(os.path.join(path, 'target_mode.pkl'))
target_return = gftIO.zload(os.path.join(path, 'target_return.pkl'))
target_risk = gftIO.zload(os.path.join(path, 'target_risk.pkl'))

asset_return = asset_return.asMatrix()
asset_weights = asset_weight.asColumnTab()
target_date = pd.to_datetime(target_date)
target_return = target_return * alpha_return
target_risk = target_risk * lambda_risk

NameError: name 'os' is not defined

In [7]:
asset_return = x3.asMatrix()
asset_weights = x4.asColumnTab()
target_date = pd.to_datetime(target_date)
target_return = target_return * alpha_return
target_risk = target_risk * lambda_risk

In [111]:
risk_model['07EFFBA4C82C1E6F228F804583D387C8'].asMatrix()

,b'\x8eK\x9c+\x8a\xc71\x00j\xfb\x07AX\x13\xc8\xac',b'\xecC\x1e\xb1\xa5\x11\x01\x01x\xa6\xc0&z\xec\xba\xb0',"b'8H\x8c\x1e\xd9\x834\x02~,\x82\xce\xfeu(\x98'",b'\xb7Hy<\xa9\xd8~\x02\x10\xcd\xa8\x1d\xc1\xed4\xa9',b'nKv\xc5%\xb5\x9f\x02\xadY/9lE\xcd\x82',b'\xb1A\x99\xa3fy\x07\x03\xe1\x0f\xdb\xf3=\xf6\xbf\xaa',"b'\x1dF""\xda&f\xee\x03RK\xf5\xe1i\xc2\x8f\x88'","b""\x08M\x0e'\x1e\xe8\xf8\x04\x89pD\x87\x80\xcf\x8d\xa1""",b'#K\xa26\xd5\x16\x1a\x057T1\xba\xb7w \xb4',b'\xe9C\x11\xfa0\x98h\x05\xc4~\xd9\x14\xa1\xce\xc5\x91',...,b'\x0eA\x0eU#g\x10\xf8\xe5\x1a\xc8i\x8b\xee:\xbd',"b'\xa0@K\xbc\xa6F,\xf8^O\xcd\xect\xfbU\x95'",b'\x1cK\xca;\xf3(\x9a\xf8m\xaf\t~R\xb6\x1e\xbf',b'@A\x953\xdf\x8e\xe3\xf8\x82\xc8\xfd\xc9:\xf0\xcb\x8c',b'NA\x03\xb0dI1\xf9\xf0\xea\xf1y\xe7\x10\xe1\xb7',"b'\xa2H\xbd\x8cZZY\xf9\xce#\xf6$""!\x05\x99'",b'bM\xcd\t|\xb5\x85\xf9u\xddi=\x89\xffO\x8d',b'\xc9M\xcaX\x11\xe2\x92\xfb\x86\xc2\rf\xcb\xe79\x9f',b'AA\xd6\xbe1\xe8:\xfd\x83D\xaa\xf3\\\r\xb0\xb6',b'\xafG\xc4\xf9\xe0\xeay\xfexD\x7f*\x85Ca\xa1'
1990-12-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-12-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-12-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-12-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
position_limit = 200
if asset_constraint is not None:
    asset_constraint = asset_constraint.asMatrix()
if group_constraint is not None:
    group_constraint = group_constraint.asMatrix()
if exposure_constraint is not None:
    exposure_constaint = exposure_constraint.asMatrix()

logger.debug('parse data finished!')
logger.debug('asset return number: %s', asset_return.shape[1])
logger.debug('asset weight number: %s', asset_weights.shape[0])
logger.debug('parse data finished.')
# regex to search all the factors
ls_factor = [x[:-4] for x in risk_model.keys() if re.search(".ret$", x)]
# ls_factor = [x.split('.')[0] for x in ls_factor]

specific_risk = risk_model['specificRisk'].pivot(
    index='date', columns='symbol', values='specificrisk')
# target_date = pd.datetime(year=2016, month=10, day=31)
# target_return = -0.00096377
# target_risk = 3.16026352e-06
#target_mode = 1


# find the nearest date next to target date from specific risk
dt_next_to_target = specific_risk.index.searchsorted(target_date)
dt_next_to_target = specific_risk.index[dt_next_to_target]
target_specific_risk = specific_risk.loc[dt_next_to_target, :]
logger.debug('target date: %s', target_date)
logger.debug('next to target date: %s', dt_next_to_target)
# drop duplicated rows at date
df_industries_asset_weight = asset_weights.drop_duplicates(
    subset=['date', 'symbol'])
try:
    df_industries_asset_init_weight = df_industries_asset_weight[
        df_industries_asset_weight['date'] == target_date].dropna()
except KeyError:
    raise KeyError('invalid input date: %s' % target_date)

# drop incomplete rows
df_industries_asset_init_weight = df_industries_asset_init_weight.dropna(
    axis=0, subset=['industry', 'symbol'], how='any')

unique_symbol = df_industries_asset_init_weight['symbol'].unique()
target_symbols = target_specific_risk.index.intersection(asset_return.columns.intersection(unique_symbol))
if position_limit > len(target_symbols):
    logger.debug("position limit is bigger than total symbols.")
    position_limit = len(target_symbols)

# get random symbols at the target position limit
arr = list(range(len(target_symbols)))
np.random.shuffle(arr)
target_symbols = target_symbols[arr[:position_limit]]

df_industries_asset_target_init_weight = df_industries_asset_init_weight.\
                                         loc[df_industries_asset_init_weight['symbol'].isin(target_symbols)]
df_pivot_industries_asset_weights = pd.pivot_table(
    df_industries_asset_target_init_weight, values='value', index=['date'],
    columns=['industry', 'symbol'])
df_pivot_industries_asset_weights = df_pivot_industries_asset_weights.fillna(0)
logger.debug("set OOTV to hierachical index dataframe.")
noa = len(target_symbols)
if noa < 1:
    raise ValueError("no intersected symbols from specific risk and initial holding.")
logger.debug("number of asset: %s", noa)
# get the ordered column list
idx_level_0_value = df_pivot_industries_asset_weights.columns.get_level_values(0)
idx_level_0_value = idx_level_0_value.drop_duplicates()
idx_level_1_value = df_pivot_industries_asset_weights.columns.get_level_values(1)
asset_return = asset_return.loc[:target_date, idx_level_1_value].fillna(0)

diag = specific_risk.loc[dt_next_to_target, idx_level_1_value]
delta = pd.DataFrame(np.diag(diag), index=diag.index,
                     columns=diag.index).fillna(0)

big_X = get_factor_exposure(risk_model, ls_factor, target_date,
                            idx_level_1_value)
big_X = big_X.fillna(0)
all_factors = big_X.columns

covariance_matrix = risk_model['ret_cov'].set_index('date')

cov_matrix = covariance_matrix.loc[dt_next_to_target]
cov_matrix = cov_matrix.pivot(index='factorid1', columns='factorid2',
                              values='value')
cov_matrix = cov_matrix.reindex(all_factors, all_factors, fill_value=np.nan)

cov_matrix_V = big_X.dot(cov_matrix).dot(big_X.T) + delta

P = matrix(cov_matrix_V.values)
q = matrix(np.zeros((noa, 1)), tc='d')

A = matrix(1.0, (1, noa))
b = matrix(1.0)

# for group weight constraint
groups = df_pivot_industries_asset_weights.groupby(
    axis=1, level=0, sort=False, group_keys=False).count().\
    iloc[-1, :].values
num_group = len(groups)
num_asset = np.sum(groups)

logger.debug('number of assets in groups: %s', groups)
logger.debug('number of groups: %s', num_group)


# set boundary vector for h
df_asset_weight = pd.DataFrame({'lower': [0.0], 'upper': [1.0]},
                               index=idx_level_1_value)
df_group_weight = pd.DataFrame({'lower': [0.0], 'upper': [1.0]},
                               index=set(idx_level_0_value))
df_factor_exposure_bound = pd.DataFrame(index=big_X.T.index, columns=[['lower', 'upper']])
df_factor_exposure_bound.lower = (1.0/noa)*big_X.sum()*(0.999991)
df_factor_exposure_bound.upper = (1.0/noa)*big_X.sum()*(1.000009)

if asset_constraint is not None:
    df_asset_weight.lower.ix[asset_constraint.lower.index] = asset_constraint.lower
    df_asset_weight.upper.ix[asset_constraint.upper.index] = asset_constraint.upper
if group_constraint is not None:
    df_group_weight.lower.ix[group_constraint.lower.index] = group_constraint.lower
    df_group_weight.upper.ix[group_constraint.upper.index] = group_constraint.upper
if exposure_constraint is not None:
    df_factor_exposure_bound.lower.ix[exposure_constraint.lower.index] = exposure_constraint.lower
    df_factor_exposure_bound.upper.ix[exposure_constraint.upper.index] = exposure_constraint.upper

if check_boundary_constraint(df_asset_weight, df_group_weight,
                             df_factor_exposure_bound, big_X):
    logger.debug("boundary setting is fine")

df_asset_bnd_matrix = matrix(np.concatenate(((df_asset_weight.upper,
                                              df_asset_weight.lower)), 0))
df_group_bnd_matrix = matrix(np.concatenate(((df_group_weight.upper,
                                              df_group_weight.lower)), 0))
df_factor_exposure_bnd_matrix = matrix(np.concatenate(((df_factor_exposure_bound.upper,
                                                        df_factor_exposure_bound.lower)), 0))

# Assuming AvgReturns as the expected returns if parameter is not specified
rets_mean = logrels(asset_return).mean()
avg_ret = matrix(rets_mean.values)
G = matrix(-np.transpose(np.array(avg_ret)))
h = matrix(-np.ones((1, 1))*target_return)
G_sparse_list = []
for i in range(num_group):
    for j in range(groups[i]):
        G_sparse_list.append(i)
Group_sub = spmatrix(1.0, G_sparse_list, range(num_asset))

Group_sub = matrix(sparse([Group_sub, -Group_sub]))

asset_sub = matrix(np.eye(noa))
asset_sub = matrix(sparse([asset_sub, -asset_sub]))
exp_sub = matrix(np.array(big_X.T))
exp_sub = matrix(sparse([exp_sub, - exp_sub]))


2017-07-04 06:48:29,771 root         DEBUG    parse data finished!
2017-07-04 06:48:29,771 root         DEBUG    parse data finished!
2017-07-04 06:48:29,776 root         DEBUG    asset return number: 200
2017-07-04 06:48:29,776 root         DEBUG    asset return number: 200
2017-07-04 06:48:29,780 root         DEBUG    asset weight number: 10540
2017-07-04 06:48:29,780 root         DEBUG    asset weight number: 10540
2017-07-04 06:48:29,785 root         DEBUG    parse data finished.
2017-07-04 06:48:29,785 root         DEBUG    parse data finished.
2017-07-04 06:48:32,328 root         DEBUG    target date: 2016-10-31 00:00:00
2017-07-04 06:48:32,328 root         DEBUG    target date: 2016-10-31 00:00:00
2017-07-04 06:48:32,331 root         DEBUG    next to target date: 2016-10-31 00:00:00
2017-07-04 06:48:32,331 root         DEBUG    next to target date: 2016-10-31 00:00:00
2017-07-04 06:48:32,365 root         DEBUG    set OOTV to hierachical index dataframe.
2017-07-04 06:48:32,365 r

In [108]:
diag

symbol
b'\xe2\x11\xeaH\x17m;\xc1"\'i\xcb+xu\x90'            0.000188
b'\xe2\x11\xeaH\x1a\xa8=\xc1"\'i\xcb+xu\x90'         0.000049
b'\xe2\x11\xeaH\x1c\xfd.\xc1"\'i\xcb+xu\x90'         0.000112
b'\xe2\x11\xeaH!\x7fA\xc1"\'i\xcb+xu\x90'            0.000212
b'\xe2\x11\xeaHV\x03<\xc1"\'i\xcb+xu\x90'            0.000105
b'\xe2\x11\xeaH[\x8e\x15\xc1"\'i\xcb+xu\x90'         0.000019
b'\xe2\x11\xeaH\x96\x94\x19\xc1"\'i\xcb+xu\x90'      0.000334
b'\xe2\x11\xeaH\x98\x89)\xc1"\'i\xcb+xu\x90'         0.000183
b'\xe2\x11\xeaH\x9c&?\xc1"\'i\xcb+xu\x90'            0.000200
b'\xe2\x11\xeaH\xad\xf3D\xc1"\'i\xcb+xu\x90'         0.000047
b'\xe2\x11\xeaH\xb1s1\xc1"\'i\xcb+xu\x90'                 NaN
b'\xe2\x11\xeaH\xe6\tA\xc1"\'i\xcb+xu\x90'           0.000028
b'\xe2\x11\xeaH\x12\xcf@\xc1"\'i\xcb+xu\x90'         0.000203
b'\xe2\x11\xeaHGN<\xc1"\'i\xcb+xu\x90'               0.000048
b'\xe2\x11\xeaH2\xa3<\xc1"\'i\xcb+xu\x90'            0.000024
b'\xe2\x11\xeaHk\xba?\xc1"\'i\xcb+xu\x90'            0.000030
b

## target mode 0
minimum risk

In [ ]:
if exposure_constraint is not None:
    G0 = matrix(sparse([asset_sub, Group_sub, exp_sub]))
    h0 = matrix(sparse([df_asset_bnd_matrix, df_group_bnd_matrix,
                       df_factor_exposure_bnd_matrix]))
else:
    G0 = matrix(sparse([asset_sub, Group_sub]))
    h0 = matrix(sparse([df_asset_bnd_matrix, df_group_bnd_matrix]))

try:
    sol = solvers.qp(P, q, G0, h0, A, b)
except ValueError:
    h = matrix(-np.ones((1, 1))*100.0)
    check_constraint_issue(P, q, G, h, A, b, asset_sub, Group_sub,
                           exp_sub, df_asset_bnd_matrix,
                           df_group_bnd_matrix,
                           df_factor_exposure_bnd_matrix)
if sol['status'] == 'unknown':
    h = matrix(-np.ones((1, 1))*100.0)
    check_constraint_issue(P, q, G, h, A, b, asset_sub, Group_sub,
                           exp_sub, df_asset_bnd_matrix,
                           df_group_bnd_matrix,
                           df_factor_exposure_bnd_matrix)
df_opts_weight = pd.DataFrame(np.array(sol['x']).T,
                              columns=target_symbols,
                              index=[target_date])
logger.debug(sol['status'])
logger.debug("target return: %s", target_return)
logger.debug("all weight are bigger than 0? %s", (df_opts_weight>0).all().all())
logger.debug("all weight are smaller than 1? %s", (df_opts_weight<=1).all().all())
logger.debug("weight sum smaller than 0: %s", df_opts_weight[df_opts_weight<0].sum(1))

df_opts_weight

In [ ]:
statistics(df_opts_weight, asset_return, cov_matrix_V)

## target mode 1
return >= target return,
minimum risk

In [114]:
(f_min, f_max) = get_ret_range(asset_return, df_asset_weight)
idx = pd.IndexSlice
df_hi_asset_return = pd.DataFrame(columns=df_pivot_industries_asset_weights.columns,
                                  index=asset_return.index)
df_hi_asset_return.loc[:, idx[:, idx_level_1_value]] = asset_return.loc[:, idx_level_1_value].values
group_rets_min_idx_level_1_value = df_hi_asset_return.mean().sort_values(ascending=True).groupby(level=0).head(1).values
group_rets_max_idx_level_1_value = df_hi_asset_return.mean().sort_values(ascending=False).groupby(level=0).head(1).values

In [ ]:
df_hi_asset_return.loc[:, idx[idx_level_0_value,:]].mean().sort_values(ascending=True).groupby(level=0).head(1)

In [115]:
group_rets_min_idx_level_1_value

array([ -3.08372332e-03,  -3.04459474e-03,  -2.47345275e-03,
        -2.46498143e-03,  -2.10531454e-03,  -2.09079794e-03,
        -1.77726270e-03,  -1.74533677e-03,  -1.60524425e-03,
        -1.59656582e-03,  -1.51261546e-03,  -1.48577200e-03,
        -1.47114596e-03,  -1.44422978e-03,  -1.37475718e-03,
        -1.27766628e-03,  -1.17392083e-03,  -1.17106165e-03,
        -1.17070168e-03,  -1.12347499e-03,  -1.10415187e-03,
        -6.29726577e-04,  -3.36962990e-04,  -2.85112202e-04,
        -2.46336174e-04,  -2.37198821e-04,  -1.97128517e-04,
         2.31517884e-05])

In [ ]:
idx_level_0_value

In [116]:
f_group_rets_min = get_ret_range(df_hi_asset_return.loc[:, group_rets_min_idx_level_1_value], df_group_weight)[0]
f_group_rets_max = get_ret_range(df_hi_asset_return.loc[:, group_rets_max_idx_level_1_value], df_group_weight)[1]

if target_return < f_min or target_return > f_max:
    raise ValueError("target return not possible")
if exposure_constraint is not None:
    G1 = matrix(sparse([G, asset_sub, Group_sub, exp_sub]))
    h1 = matrix(sparse([h, df_asset_bnd_matrix, df_group_bnd_matrix,
                       df_factor_exposure_bnd_matrix]))
else:
    G1 = matrix(sparse([G, asset_sub, Group_sub]))
    h1 = matrix(sparse([h, df_asset_bnd_matrix, df_group_bnd_matrix]))
try:
    sol = solvers.qp(P, q, G1, h1, A, b)
except ValueError:
    check_constraint_issue(P, q, G, h, A, b, asset_sub, Group_sub,
                           exp_sub, df_asset_bnd_matrix,
                           df_group_bnd_matrix,
                           df_factor_exposure_bnd_matrix)
if sol['status'] == 'unknown':
    check_constraint_issue(P, q, G, h, A, b, asset_sub, Group_sub,
                           exp_sub, df_asset_bnd_matrix,
                           df_group_bnd_matrix,
                           df_factor_exposure_bnd_matrix)

df_opts_weight = pd.DataFrame(np.array(sol['x']).T,
                              columns=target_symbols,
                              index=[target_date])
logger.debug(sol['status'])
logger.debug("target return: %s", target_return)
logger.debug("all weight are bigger than 0? %s", (df_opts_weight>0).all().all())
logger.debug("all weight are smaller than 1? %s", (df_opts_weight<=1).all().all())
logger.debug("weight sum smaller than 0: %s", df_opts_weight[df_opts_weight<0].sum(1))

df_opts_weight

ValueError: sum of total asset maximum weight is less than 1 

In [ ]:
statistics(df_opts_weight, asset_return, cov_matrix_V)

### target mode 2
risk <= target risk, 
maximum sharpe ratio

##### monte carlo

In [118]:
(f_min, f_max) = get_ret_range(asset_return, df_asset_weight)
N = 20
f_step = (f_max - f_min) / N
ls_f_return = [f_min + x * f_step for x in range(N + 1)]
ls_f_risk = []
ls_portfolio = []
ls_f_result = []
for i, f_target_return in enumerate(ls_f_return):
    logger.debug('target return: %s %s', i, f_target_return)
    h = matrix(-np.ones((1, 1))*f_target_return)
    if exposure_constraint is not None:
        G_sr = matrix(sparse([G, asset_sub, Group_sub, exp_sub]))
        h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                              df_group_bnd_matrix,
                              df_factor_exposure_bnd_matrix]))
    else:
        G_sr = matrix(sparse([G, asset_sub, Group_sub]))
        h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                              df_group_bnd_matrix]))
    try:
        sol = solvers.qp(P, q, G_sr, h_sr, A, b)
        logger.debug("solution is: %s", sol['status'])
    except:
        ls_f_risk.append(np.nan)
        ls_portfolio.append(None)
        continue
    f_result = statistics(sol['x'], asset_return, cov_matrix_V)
    ls_f_result.append(f_result)
    logger.debug('target result calculated from weight: %s', f_result)

    ls_f_risk.append(statistics(sol['x'], asset_return, cov_matrix_V)[1])
    df_opts_weight = pd.DataFrame(np.array(sol['x']).T,
                                  columns=target_symbols,
                                  index=[target_date])
    ls_portfolio.append(df_opts_weight)

ls_f_return_new = np.array(ls_f_return)
ls_f_risk_new = np.array(ls_f_risk)
ls_f_risk_new = ls_f_risk_new[ls_f_risk_new <= target_risk]
if len(ls_f_risk_new) == 0:
    raise ValueError("target risk is not possible")
ls_f_return_new = ls_f_return_new[ls_f_risk_new <= target_risk]
na_sharpe_ratio = np.array(ls_f_result)[:, 2]
logger.debug("maximum sharpe ratio is: %s", max(na_sharpe_ratio))
i_index_max_sharpe = np.where(na_sharpe_ratio == max(na_sharpe_ratio))
i_index_max_sharpe = i_index_max_sharpe[0]
f_target = ls_f_return_new[i_index_max_sharpe]
h = matrix(-np.ones((1, 1))*f_target)
logger.debug("maximum sharpe ratio index is: %s", i_index_max_sharpe)
logger.debug("maximum sharpe ratio return is: %s", f_target)
logger.debug("maximum sharpe ratio risk is: %s", ls_f_risk_new[i_index_max_sharpe])

if exposure_constraint is not None:
    G_sr = matrix(sparse([G, asset_sub, Group_sub, exp_sub]))
    h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                          df_group_bnd_matrix,
                          df_factor_exposure_bnd_matrix]))
else:
    G_sr = matrix(sparse([G, asset_sub, Group_sub]))
    h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                          df_group_bnd_matrix]))
sol = solvers.qp(P, q, G_sr, h_sr, A, b)
logger.debug("maximum sharpe ratio solution is %s", sol['status'])
df_opts_weight = pd.DataFrame(np.array(sol['x']).T,
                              columns=target_symbols,
                              index=[target_date])

2017-07-04 06:48:47,708 root         DEBUG    target return: 0 -0.00377945858449
2017-07-04 06:48:47,708 root         DEBUG    target return: 0 -0.00377945858449
2017-07-04 06:48:47,779 root         DEBUG    solution is: optimal
2017-07-04 06:48:47,779 root         DEBUG    solution is: optimal
2017-07-04 06:48:47,793 root         DEBUG    target result calculated from weight: [-0.0008199889833122861, array([[  1.10342121e-08]]), array([[-7.80615829]])]
2017-07-04 06:48:47,793 root         DEBUG    target result calculated from weight: [-0.0008199889833122861, array([[  1.10342121e-08]]), array([[-7.80615829]])]
2017-07-04 06:48:47,811 root         DEBUG    target return: 1 -0.00322795667782
2017-07-04 06:48:47,811 root         DEBUG    target return: 1 -0.00322795667782
2017-07-04 06:48:47,877 root         DEBUG    solution is: optimal
2017-07-04 06:48:47,877 root         DEBUG    solution is: optimal
2017-07-04 06:48:47,891 root         DEBUG    target result calculated from weight: 

In [ ]:
f_target = ls_f_return_new[i_index_max_sharpe]
h = matrix(-np.ones((1, 1))*0.0762)

if exposure_constraint is not None:
    G_sr = matrix(sparse([G, asset_sub, Group_sub, exp_sub]))
    h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                          df_group_bnd_matrix,
                          df_factor_exposure_bnd_matrix]))
else:
    G_sr = matrix(sparse([G, asset_sub, Group_sub]))
    h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                          df_group_bnd_matrix]))
sol = solvers.qp(P, q, G_sr, h_sr, A, b)
logger.debug("maximum sharpe ratio solution is %s", sol['status'])
df_opts_weight = pd.DataFrame(np.array(sol['x']).T,
                              columns=target_symbols,
                              index=[target_date])

#### SOCP

In [ ]:
(f_min, f_max) = get_ret_range(asset_return, df_asset_weight)
N = 20
f_step = (f_max - f_min) / N
ls_f_return = [f_min + x * f_step for x in range(N + 1)]
ls_f_risk = []
ls_portfolio = []
ls_f_result = []
for i, f_target_return in enumerate(ls_f_return):
    logger.debug('target return: %s %s', i, f_target_return)
    h = matrix(-np.ones((1, 1))*f_target_return)
    if exposure_constraint is not None:
        G_sr = matrix(sparse([G, asset_sub, Group_sub, exp_sub]))
        h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                              df_group_bnd_matrix,
                              df_factor_exposure_bnd_matrix]))
    else:
        G_sr = matrix(sparse([G, asset_sub, Group_sub]))
        h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                              df_group_bnd_matrix]))
    try:
        sol = solvers.qp(P, q, G_sr, h_sr, A, b)
        logger.debug("solution is: %s", sol['status'])
    except:
        ls_f_risk.append(np.nan)
        ls_portfolio.append(None)
        continue
    f_result = statistics(sol['x'], asset_return, cov_matrix_V)
    ls_f_result.append(f_result)
    logger.debug('target result calculated from weight: %s', f_result)

    ls_f_risk.append(statistics(sol['x'], asset_return, cov_matrix_V)[1])
    df_opts_weight = pd.DataFrame(np.array(sol['x']).T,
                                  columns=target_symbols,
                                  index=[target_date])
    ls_portfolio.append(df_opts_weight)

ls_f_return_new = np.array(ls_f_return)
ls_f_risk_new = np.array(ls_f_risk)
ls_f_risk_new = ls_f_risk_new[ls_f_risk_new <= target_risk]
if len(ls_f_risk_new) == 0:
    raise ValueError("target risk is not possible")
ls_f_return_new = ls_f_return_new[ls_f_risk_new <= target_risk]
na_sharpe_ratio = np.array(ls_f_result)[:, 2]
logger.debug("maximum sharpe ratio is: %s", max(na_sharpe_ratio))
i_index_max_sharpe = np.where(na_sharpe_ratio == max(na_sharpe_ratio))
i_index_max_sharpe = i_index_max_sharpe[0]
f_target = ls_f_return_new[i_index_max_sharpe]
h = matrix(-np.ones((1, 1))*f_target)
logger.debug("maximum sharpe ratio index is: %s", i_index_max_sharpe)
logger.debug("maximum sharpe ratio return is: %s", f_target)
logger.debug("maximum sharpe ratio risk is: %s", ls_f_risk_new[i_index_max_sharpe])

if exposure_constraint is not None:
    G_sr = matrix(sparse([G, asset_sub, Group_sub, exp_sub]))
    h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                          df_group_bnd_matrix,
                          df_factor_exposure_bnd_matrix]))
else:
    G_sr = matrix(sparse([G, asset_sub, Group_sub]))
    h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                          df_group_bnd_matrix]))
sol = solvers.qp(P, q, G_sr, h_sr, A, b)
logger.debug("maximum sharpe ratio solution is %s", sol['status'])
df_opts_weight = pd.DataFrame(np.array(sol['x']).T,
                              columns=target_symbols,
                              index=[target_date])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(8, 4))
plt.scatter(ls_f_risk, ls_f_return,
            c=np.array(ls_f_result)[:, 2], marker='o')
            # random portfolio composition

plt.grid(True)
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.colorbar(label='Sharpe ratio')

#### SOCP

In [ ]:
(f_min, f_max) = get_ret_range(asset_return, df_asset_weight)
N = 20
f_step = (f_max - f_min) / N
ls_f_return = [f_min + x * f_step for x in range(N + 1)]
ls_f_risk = []
ls_portfolio = []
ls_f_result = []
for i, f_target_return in enumerate(ls_f_return):
    logger.debug('target return: %s %s', i, f_target_return)
    h = matrix(-np.ones((1, 1))*f_target_return)
    if exposure_constraint is not None:
        G_sr = matrix(sparse([G, asset_sub, Group_sub, exp_sub]))
        h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                              df_group_bnd_matrix,
                              df_factor_exposure_bnd_matrix]))
    else:
        G_sr = matrix(sparse([G, asset_sub, Group_sub]))
        h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                              df_group_bnd_matrix]))
    try:
        sol = solvers.qp(P, q, G_sr, h_sr, A, b)
        logger.debug("solution is: %s", sol['status'])
    except:
        ls_f_risk.append(np.nan)
        ls_portfolio.append(None)
        continue
    f_result = statistics(sol['x'], asset_return, cov_matrix_V)
    ls_f_result.append(f_result)
    logger.debug('target result calculated from weight: %s', f_result)

    ls_f_risk.append(statistics(sol['x'], asset_return, cov_matrix_V)[1])
    df_opts_weight = pd.DataFrame(np.array(sol['x']).T,
                                  columns=target_symbols,
                                  index=[target_date])
    ls_portfolio.append(df_opts_weight)

ls_f_return_new = np.array(ls_f_return)
ls_f_risk_new = np.array(ls_f_risk)
ls_f_risk_new = ls_f_risk_new[ls_f_risk_new <= target_risk]
if len(ls_f_risk_new) == 0:
    raise ValueError("target risk is not possible")
ls_f_return_new = ls_f_return_new[ls_f_risk_new <= target_risk]
na_sharpe_ratio = np.array(ls_f_result)[:, 2]
logger.debug("maximum sharpe ratio is: %s", max(na_sharpe_ratio))
i_index_max_sharpe = np.where(na_sharpe_ratio == max(na_sharpe_ratio))
i_index_max_sharpe = i_index_max_sharpe[0]
f_target = ls_f_return_new[i_index_max_sharpe]
h = matrix(-np.ones((1, 1))*f_target)
logger.debug("maximum sharpe ratio index is: %s", i_index_max_sharpe)
logger.debug("maximum sharpe ratio return is: %s", f_target)
logger.debug("maximum sharpe ratio risk is: %s", ls_f_risk_new[i_index_max_sharpe])

if exposure_constraint is not None:
    G_sr = matrix(sparse([G, asset_sub, Group_sub, exp_sub]))
    h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                          df_group_bnd_matrix,
                          df_factor_exposure_bnd_matrix]))
else:
    G_sr = matrix(sparse([G, asset_sub, Group_sub]))
    h_sr = matrix(sparse([h, df_asset_bnd_matrix,
                          df_group_bnd_matrix]))
sol = solvers.qp(P, q, G_sr, h_sr, A, b)
logger.debug("maximum sharpe ratio solution is %s", sol['status'])
df_opts_weight = pd.DataFrame(np.array(sol['x']).T,
                              columns=target_symbols,
                              index=[target_date])

In [133]:
# Compute A matrix in optimization
# A is the square root of SIGMA
U,V = np.linalg.eig(cov_matrix_V)
# Project onto PSD
U[U<0] = 0
Usqrt = np.sqrt(U)
A = np.dot(np.diag(Usqrt),V.T)

In [134]:
for i in np.arange(noa):
    ei = np.zeros((1,noa))
    ei[0,i] = 1
    if i == 0:
        G2temp = [matrix(-ei)]
        h2temp = [matrix(np.zeros((1,1)))]
    else:
        G2temp += [matrix(-ei)]
        h2temp += [matrix(np.zeros((1,1)))]

In [135]:
# Generate G and h matrices
G1temp = np.zeros((A.shape[0]+1,A.shape[1]))
G1temp[1:,:] = -A
#G1 = matrix(G1temp)
h1temp = np.zeros((A.shape[0]+1,1))
h1temp[0] = np.sqrt(target_risk)
#h1 = matrix(h1temp)

/home/gft/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: ComplexWarning: Casting complex values to real discards the imaginary part
  This is separate from the ipykernel package so we can avoid doing imports until


In [128]:
G2temp = matrix(-np.eye(noa))
h2temp = matrix(np.zeros((noa, 1)))

In [15]:
print(G.T)

[ 9.97e-04]
[ 8.82e-04]
[-2.01e-04]
[ 1.08e-03]
[ 7.60e-04]
[ 9.32e-04]
[ 2.11e-03]
[ 1.21e-03]
[ 1.25e-03]
[ 4.79e-04]
[ 3.75e-03]
[ 1.05e-03]
[ 3.58e-04]
[ 2.58e-03]
[ 1.02e-03]
[ 8.09e-04]
[ 8.85e-04]
[ 5.79e-04]
[ 1.98e-03]
[ 4.40e-04]
[ 7.02e-04]
[ 1.96e-03]
[ 5.65e-04]
[-6.39e-04]
[ 2.09e-03]
[ 3.62e-04]
[ 8.77e-04]
[-2.04e-03]
[ 1.20e-03]
[-1.14e-05]
[ 9.77e-04]
[ 1.54e-03]
[ 1.72e-03]
[-5.55e-03]
[-2.33e-03]
[ 1.98e-03]
[-4.85e-03]
[ 2.02e-03]
[ 1.96e-03]
[ 5.86e-04]
[ 9.99e-04]
[ 5.04e-04]
[ 1.12e-03]
[ 1.90e-03]
[ 1.28e-03]
[ 2.13e-03]
[ 3.04e-03]
[ 1.90e-03]
[ 8.37e-04]
[ 7.42e-04]
[ 2.06e-04]
[ 1.94e-03]
[ 1.36e-03]
[ 1.28e-03]
[ 1.28e-03]
[ 1.11e-03]
[ 7.45e-04]
[ 1.01e-03]
[ 2.38e-03]
[ 5.88e-04]
[ 2.87e-03]
[ 1.80e-03]
[-3.71e-03]
[ 7.33e-04]
[ 2.42e-03]
[ 2.47e-03]
[ 2.01e-03]
[ 2.07e-03]
[ 3.28e-03]
[ 2.17e-03]
[ 1.26e-03]
[ 3.46e-03]
[ 1.87e-03]
[ 7.03e-04]
[ 1.94e-03]
[-3.69e-03]
[ 1.58e-03]
[ 2.05e-03]
[ 2.08e-03]
[ 3.34e-04]
[ 5.23e-05]
[ 7.20e-04]
[ 3.78e-03]
[ 2.

In [53]:
A

(200, 200)

In [129]:
F = matrix(np.ones((1, noa)))
g = matrix(np.ones((1, 1)))

In [130]:
Gq = [matrix(G1temp)] + G2temp
hq = [matrix(h1temp)] + h2temp

TypeError: can only concatenate list (not "cvxopt.base.matrix") to list

In [85]:
print(h1)

[ 1.78e-03]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.

In [131]:
sol = solvers.socp(c=G.T, Gq=Gq, hq=hq, A=F, b=g)

[<201x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 matrix, tc='d'>,
 <1x1 mat

In [132]:
statistics(sol['x'], asset_return, cov_matrix_V)

[0.0023531876543094517, array([[  3.16026198e-06]]), array([[ 1.32371661]])]

In [100]:
Gq = matrix(sparse([G1temp, G2temp]))
hq = matrix(sparse([h1temp, h2temp]))

TypeError: invalid type in list

In [26]:
print(-np.transpose(np.array(avg_ret)))

[[  9.97054457e-04   8.82337749e-04  -2.00998623e-04   1.07878765e-03
    7.59579712e-04   9.31785371e-04   2.11097446e-03   1.20822673e-03
    1.24943312e-03   4.78877728e-04   3.74847443e-03   1.05323892e-03
    3.58007484e-04   2.57625490e-03   1.02265845e-03   8.08522628e-04
    8.84783957e-04   5.78786514e-04   1.97699158e-03   4.39976042e-04
    7.01927439e-04   1.95570807e-03   5.65489011e-04  -6.38787221e-04
    2.08809331e-03   3.62173214e-04   8.76676538e-04  -2.03524755e-03
    1.19643142e-03  -1.13687151e-05   9.77458698e-04   1.53952840e-03
    1.72037390e-03  -5.54561855e-03  -2.32561040e-03   1.98418450e-03
   -4.84742224e-03   2.01697072e-03   1.96098732e-03   5.86318711e-04
    9.99488551e-04   5.04065746e-04   1.11860747e-03   1.89850983e-03
    1.27742485e-03   2.12914510e-03   3.03509698e-03   1.90380442e-03
    8.37313789e-04   7.41544937e-04   2.05916119e-04   1.94108896e-03
    1.35904646e-03   1.27724190e-03   1.28277338e-03   1.10881721e-03
    7.44561711e-04  

In [51]:
# Generate covariance matrix
counter = 0
numPOS = pbar.size
SIGMA = np.zeros((numPOS,numPOS))
for i in np.arange(numPOS-1):
    for j in np.arange(i,numPOS-1):
        if i == j:
            SIGMA[i,j] = varvec[i]
        else:
            SIGMA[i,j] = covvec[counter]
            SIGMA[j,i] = SIGMA[i,j]
            counter+=1

# Compute A matrix in optimization
# A is the square root of SIGMA
U,V = np.linalg.eig(SIGMA)
# Project onto PSD
U[U<0] = 0
Usqrt = np.sqrt(U)
A = np.dot(np.diag(Usqrt),V.T)

# Generate G and h matrices
G1temp = np.zeros((A.shape[0]+1,A.shape[1]))
G1temp[1:,:] = -A
h1temp = np.zeros((A.shape[0]+1,1))
h1temp[0] = np.sqrt(varmax)	

for i in np.arange(numPOS):
    ei = np.zeros((1,numPOS))
    ei[0,i] = 1
    if i == 0:
        G2temp = [matrix(-ei)]
        h2temp = [matrix(np.zeros((1,1)))]
    else:
        G2temp += [matrix(-ei)]
        h2temp += [matrix(np.zeros((1,1)))]

# Construct list of matrices
Ftemp = np.ones((1,numPOS))
F = matrix(Ftemp)
g = matrix(np.ones((1,1)))

G = [matrix(G1temp)] + G2temp
H = [matrix(h1temp)] + h2temp

# Solve QCQP
# solvers.options['show_progress'] = False
# Passing in -matrix(pbar) since maximizing
sol = solvers.socp(-matrix(pbar),Gq=G,hq=H,A=F,b=g)
xsol = np.array(sol['x'])


ValueError: setting an array element with a sequence.

In [106]:
print(sol['x'])

[ 3.18e-09]
[ 4.05e-09]
[ 7.41e-09]
[ 3.15e-09]
[ 3.81e-09]
[ 3.66e-09]
[ 2.04e-09]
[ 3.51e-09]
[ 3.77e-09]
[ 7.08e-09]
[ 1.35e-09]
[ 4.37e-09]
[ 6.18e-09]
[ 1.58e-09]
[ 5.48e-09]
[ 8.68e-09]
[ 7.02e-09]
[ 6.12e-09]
[ 1.82e-09]
[ 4.65e-09]
[ 3.11e-09]
[ 1.86e-09]
[ 3.74e-09]
[ 4.71e-09]
[ 1.87e-09]
[ 2.44e-09]
[ 4.98e-09]
[ 2.21e-04]
[ 2.68e-09]
[ 3.89e-01]
[ 3.50e-09]
[ 2.05e-09]
[ 2.18e-09]
[ 1.01e-01]
[ 4.44e-02]
[ 2.03e-09]
[ 5.01e-02]
[ 1.85e-09]
[ 2.19e-09]
[ 3.40e-09]
[ 2.82e-09]
[ 4.67e-09]
[ 2.91e-09]
[ 1.95e-09]
[ 2.13e-09]
[ 2.05e-09]
[ 1.57e-09]
[ 1.91e-09]
[ 4.06e-09]
[ 3.24e-09]
[ 7.67e-09]
[ 2.37e-09]
[ 2.72e-09]
[ 3.38e-09]
[ 1.89e-09]
[ 1.96e-09]
[ 3.89e-09]
[ 4.23e-09]
[ 1.78e-09]
[ 4.19e-09]
[ 1.30e-09]
[ 2.34e-09]
[ 2.33e-04]
[ 4.24e-09]
[ 1.62e-09]
[ 1.39e-09]
[ 2.16e-09]
[ 2.29e-09]
[ 1.58e-09]
[ 2.06e-09]
[ 2.50e-09]
[ 1.34e-09]
[ 1.59e-09]
[ 5.60e-09]
[ 2.32e-09]
[ 5.86e-02]
[ 3.67e-09]
[ 2.31e-09]
[ 2.33e-09]
[ 2.38e-09]
[ 1.20e-08]
[ 2.91e-09]
[ 1.42e-09]
[ 1.

In [80]:
G

<1x200 matrix, tc='d'>

In [50]:
varvec.shape

(200,)

In [48]:
# Generate mean return vector
pbar = np.append(meanvec,irate)
# Get Values from varvec and covvec
varvec = varvec.values
covvec = covvec.values

In [45]:
meanvec = np.transpose(np.array(rets_mean))
varvec = logrels(asset_return).var()
covvec = logrels(asset_return).cov()
irate = target_return
varmax = logrels(asset_return).cov()

In [23]:
print(G2temp)

[<1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, tc='d'>, <1x200 matrix, 

[<1x4 matrix, tc='d'>]
[<1x4 matrix, tc='d'>, <1x4 matrix, tc='d'>]
[<1x4 matrix, tc='d'>, <1x4 matrix, tc='d'>, <1x4 matrix, tc='d'>]
[<1x4 matrix, tc='d'>, <1x4 matrix, tc='d'>, <1x4 matrix, tc='d'>, <1x4 matrix, tc='d'>]


In [18]:
G.size

(1, 200)

In [13]:
dd

[[ 0.00177771]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.     